# Introduction to Machine Learning in Astronomy: Part 1
Written by Aayush Arya (version: Aug 27, 2023)

See the accompanying video lecture for the technical background

![Classification](Classification_Clustering.png)

This problem seems trivial, but has a lot of useful lessons to illustrate. 

The $x$ and $y$ quantities could be phase space coordinates, or color index/magnitude. 
We know that in a Hertzsprung-Russell diagram, stars in different evolutionary phases "clump" in different regions. There is indeed then a correlation between position in the HR diagram and luminosity class/evolutionary phase.

![HR Diagram](https://upload.wikimedia.org/wikipedia/commons/6/6b/HRDiagram.png)
Image: Wikipedia

You have been provided with three files: `category_dataset.csv` contains a bunch of $(x,y)$ coordinates with each point having an associated _category_ (in abstract terms, either 0 or 1). You will use this as a dataset for training your classifier.

Then, we'll look at what happens if you ask your network to make a prediction on an $(x,y)$ pair that is very different from what it's already seen (`unseen_test.csv`). 

Finally, I will ask you to train with a dataset that contains both of $(x,y)$ and see how the model's predictability changes further(`extended_training.csv`).

In [ ]:
import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense

# A method that lets you easily split a dataset into "training" and "test" subsets
from sklearn.model_selection import train_test_split

In [ ]:
plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.serif'] = ['Computer Modern']

In [ ]:
dataset = pd.read_csv('./data/category_dataset.csv')
dataset

You may be wondering what this dataset is

![Block structure](./true_distrib.png)

In [ ]:
coords = dataset.drop('category', axis=1) # input
labels = dataset['category'] # true output

coords

You need a `Model` class for the network architecture. The class [Sequential](https://keras.io/api/models/sequential/) in `keras` gives you the bones for a simple one that is going to be relevant for us. `Sequential` because one layer is passed on to after the other. For our _feedforward_ or _fully connected_ neural network, this is going to be all we need.

In [ ]:
network = Sequential()

network.add(keras.Input(shape=(2,)))
network.add(Dense(6, activation='relu'))
network.add(Dense(8, activation='relu'))

network.add(Dense(1, activation='sigmoid'))

In [ ]:
network.compile(optimizer='adam', loss=keras.losses.BinaryCrossentropy(from_logits=False),
                metrics=['accuracy', keras.metrics.BinaryAccuracy()])

The optimizer's role is to find an optimal path to minimizing the loss function. ADAM is a superior choice (compared to Stochastic Gradient Descent). We encourage to figure out why! (Hint: It's ADAptive. You should read about vanishing/exploding gradients problem.)

In [ ]:
network.summary()

Apparently, the network summary doesn't show the input shape.

In [ ]:
history = network.fit(coords, labels, batch_size=250, epochs=100)

In [ ]:
history.history.keys()

In [ ]:
fig2, ax2 = plt.subplots(1, 2, figsize=(12,6))

ax2[1].plot(history.history['binary_accuracy'])
ax2[0].plot(history.history['loss'])
for ax in ax2:
    ax.set_xlabel('Epoch')
ax2[1].set_ylabel('Binary Accuracy')
ax2[0].set_ylabel('Loss Function - $\mathcal{L}$')

I want to see what kind of predictions the network makes across the whole range of $x,y \in [-1, 1]$. Plotting that would make it easy to see what the model is actually learning.

In [ ]:
def make_square_grid(min_val=-2, max_val=2):
    x = np.linspace(min_val, max_val, 100)
    y = x
    XX, YY = np.meshgrid(x, y, indexing='xy')

    xx = XX.flatten()
    yy = YY.flatten()
    return pd.DataFrame(data = {'x': xx, 'y': yy})

In [ ]:
grid_to_evaluate = make_square_grid(-1, 1)

grid_to_evaluate

In [ ]:
predictions = network.predict(grid_to_evaluate)

I also like to test a few examples individually for personal satisfaction, lol

In [ ]:
lol = network.predict(np.array([[0.5, 0.5], [-0.5, 0.5]]))

In [ ]:
lol

In [ ]:
np.unique(predictions)

In [ ]:
grid_vals = predictions.reshape((100, 100))
#img = XX[:,:, np.newaxis]
#img[:,:,0] = grid_vals

In [ ]:
plt.imshow(grid_vals, cmap='RdYlBu')

So far so good. But how well does a network perform when it comes to making predictions for input that differs from the training data.

In [ ]:
larger = make_square_grid(-2, 2)

Data points from the true distribution that I didn't include in the first dataset.

In [ ]:
unseen_data = pd.read_csv('./data/unseen_test.csv')

In [ ]:
grid_vals2 = network.predict(larger).reshape((100, 100))

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].scatter(unseen_data['x'], unseen_data['y'], c=unseen_data['category'], ec='k', cmap='RdYlBu')
axes[1].imshow(grid_vals2, cmap='RdYlBu', extent=(-2, 2, -2, 2))
plt.tight_layout()

It seems that that the model expected the same behavior to continue.

Here we had just two 2 coordinates and two possible categories. How wrong could the model go if we had hundred input variables and 15 different categories? 

In [ ]:
unseen_coords = unseen_data.drop('category', axis=1)
unseen_labels = unseen_data['category']

Let's try training once again, but only with the _outer_ region, cutting out the inner $x, y \in [-1, 1]$

In [ ]:
hist2 = network.fit(unseen_coords, unseen_labels, batch_size=250, epochs=50)

In [ ]:
new_pred = network.predict(unseen_coords)

In [ ]:
fig2, ax2 = plt.subplots(figsize=(6,6))
plt.scatter(unseen_coords['x'], unseen_coords['y'], c=new_pred, ec='w', lw=0.4, cmap='RdYlBu')

I wonder how intact is the _inner_ region. Let's see for a second

In [ ]:
new_prediction = network.predict(larger).reshape((100,100))

plt.imshow(new_prediction, cmap='RdYlBu', extent=(-2, 2, -2, 2))

This may at first seem demotivating as to how unreliable our (rather simple) neural network is for such a trivial classification task. I would remark that some neural network architectures are more suitable for learning different kinds of information. For example, convolutional neural networks are very useful in image recognition. If you have the image of a handwritten "6", a CNN can pick up its pattern and identify it even if you rotate the image for instance.

Note that you aren't ever restricted to using _just_ a convolutional architecture or a recurrent neural network or some other _named_ class. A real problem may require a combination or even a custom design. 

One of the recent, most powerful use of machine learning in science has been [AlphaFold](https://www.nature.com/articles/s41586-021-03819-2), which was a highlighted breakthrough in protein structure prediction. Look at how specialized their architecture is.  

## Questions:
1. If you actually were to implement such a method for classifying stars, what other things apart from $B-V$ and $V$ would you consider to diminish wrong classifications? (Whether it be additional info from photometry, or any physically motivated idea)
2. Can you guess why ReLU is often a superior activation function compared to sigmoid, for example? Also, could the ReLU be improved? (Hint: Yes!)
3. Is machine learning the optimal solution for this problem?

## Homework:
Take the `extended_training.csv` dataset. It contains $(x,y)$ values covering both the regions we encountered. Train the network with it from scratch. Now try to see what the network predicts for values extending even further out, i.e. $(-3,-3)$ to $(3, 3)$. Does it pick up the recurring pattern this time? Do the results suprise you?

Try truncating the training at 20 epochs or less. Does that help? Were we just overfitting?

If you want to try things for fun to deepen your understanding: What's the effect of changing the number of hidden layers or neurons within a single layer. Intuitively, how is adding more cells to a layer different from more layers in a sequence? (Think of the algebra, maybe?)

## Additional Resources

* [Deep Learning for Physics Research](https://www.google.de/books/edition/Deep_Learning_For_Physics_Research/8dM3EAAAQBAJ?hl=en&gbpv=1&printsec=frontcover)

* Lecture series [Machine Learning for Physicists](https://youtube.com/playlist?list=PLemsnf33Vij4eFWwtoQCrt9AHjLe3uo9_&si=INosux5cpQhzNs-5) taught by Florian Marquardt 
    * Relevant Jupyter notebooks can be found at [the associated GitHub repository](https://github.com/FlorianMarquardt/machine-learning-for-physicists) 